In [1]:
#conda install -c conda-forge librosa
#conda install -c conda-forge ffmpeg
import librosa
import numpy as np 
import librosa.display
import matplotlib.pyplot as plt
import pickle
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import urllib
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from os import listdir
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler, MinMaxScaler

# Functions to get features of each song and average them by album

In [2]:
cid = '9dc2d5b21f764a068bd348f3bca570ce'
secret = 'dc76152cb84140b293645c92d09e3996'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_Album_Averages(recording_mbids):
    
    chroma_stft_ = []
    spec_cent_ = []
    spec_bw_ = []
    rolloff_ = []
    zcr_ = []
    mfcc_ = []
    
    for recording_mbid in recording_mbids:
        #print(recording_mbid)
        recording_mbid_url = 'https://musicbrainz.org/recording/' + recording_mbid
        raw_html = simple_get(recording_mbid_url)
        html = BeautifulSoup(raw_html, 'html.parser')
        
        for p in html.select('title'):
            title = str(p.text)
        #print(title)
        
        bad_chars = [';', ':', '!', "*", "?", "/", "'", "\""] 
        
        track_start = len("Recording") + 2
        track_stop = title.find("by") - 2
        trackName = title[track_start:track_stop]
        
        if "’" in trackName: 
            trackName = trackName.replace("’", "'")
            
        for i in bad_chars : 
            trackName = trackName.replace(i, '') 
            
        #print(trackName)
        
        artist_start = title.find("by") + 3
        artist_stop = title.find("-") - 1
        artistName = title[artist_start:artist_stop]

        if "’" in artistName: 
            artistName = trackName.replace("’", "'")
            
        for i in bad_chars : 
            artistName = artistName.replace(i, '') 
            
        #print(artistName)
                                          
        song_status = get_mp3(trackName, artistName)
        
        if song_status == "Song found":
            
            #print(song_status)
        
            chroma_stft, spec_cent, spec_bw, rolloff, zcr, mfcc = extract_features(trackName)

            chroma_stft_.append(chroma_stft)
            spec_cent_.append(spec_cent)
            spec_bw_.append(spec_bw)
            rolloff_.append(rolloff)
            zcr_.append(zcr)
            mfcc_.append(mfcc)
            
        else:
            print(song_status)
    
    #print("starting to average")
    if chroma_stft_ != []:
        mfcc_1 = []
        mfcc_2 = []
        mfcc_3 = []
        mfcc_4 = []
        mfcc_5 = []

        for i in mfcc_:
            mfcc_1.append(i[0])
            mfcc_2.append(i[1])
            mfcc_3.append(i[2])
            mfcc_4.append(i[3])
            mfcc_5.append(i[4])

        chroma_stft_avg = sum(chroma_stft_)/len(chroma_stft_)
        spec_cent_avg = sum(spec_cent_)/len(spec_cent_)
        spec_bw_avg = sum(spec_bw_)/len(spec_bw_)
        rolloff_avg = sum(rolloff_)/len(rolloff_)
        zcr_avg = sum(zcr_)/len(zcr_)
        mfcc_1_avg = sum(mfcc_1)/len(mfcc_1)
        mfcc_2_avg = sum(mfcc_2)/len(mfcc_2)
        mfcc_3_avg = sum(mfcc_3)/len(mfcc_3)
        mfcc_4_avg = sum(mfcc_4)/len(mfcc_4)
        mfcc_5_avg = sum(mfcc_5)/len(mfcc_5)

        album_averages = [chroma_stft_avg, spec_cent_avg, spec_bw_avg, rolloff_avg, zcr_avg, mfcc_1_avg, mfcc_2_avg, mfcc_3_avg, mfcc_4_avg, mfcc_5_avg]
        
    else:
        
        album_averages = []

    return album_averages

def get_mp3(trackName, artistName):
    search_string = trackName + " " + artistName
    results = sp.search(q= search_string, limit=10, offset=0, type='track', market=None)
    song_results = results['tracks']['items']
    
    if song_results != []:
        #print("found the song!")
        song = song_results[0]
        preview_url = song['preview_url']
        #print(preview_url)
        
        if preview_url != None:
            urllib.request.urlretrieve (preview_url, "mp3s/" + trackName + ".mp3")
            return "Song found"
        
        else: 
            return "Preview not found"
        
    else: 
        return "Song not found"
    
def extract_features(trackName):
   
    y, sr = librosa.load("mp3s/"+ trackName + ".mp3", duration = 15)
    chroma_stft = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
    spec_cent = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    zcr = np.mean(librosa.feature.zero_crossing_rate(y))
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 5)
    
    mfcc_coef = []
    for c in mfcc:
        mfcc_coef.append(np.mean(c))
             
    return chroma_stft, spec_cent, spec_bw, rolloff, zcr, mfcc_coef

# Transform MuMu Dataset

In [4]:
infile = open('dfTracks.pkl','rb')
track_dataset = pickle.load(infile)
infile.close()

In [5]:
print(track_dataset)

         amazon_id                            album_mbid        MSD_track_id  \
0       B0018ZB6ZO  0164b5ce-42f8-46bf-b140-61ecd2ed449e  TRYDUXK12903CE0C37   
1       B0018ZB6ZO  0164b5ce-42f8-46bf-b140-61ecd2ed449e  TRGWXYX128F426E27F   
2       B000003HGR  7fe6e337-9115-3719-855b-0441c42a2c36  TRVMIIJ12903CDBF1D   
3       B000003HGR  7fe6e337-9115-3719-855b-0441c42a2c36  TRDHJVA128F9335634   
4       B000003HGR  7fe6e337-9115-3719-855b-0441c42a2c36  TRKDLHL12903CDBF32   
...            ...                                   ...                 ...   
159377  B00004NH92  31d692d2-c9a5-3637-bc59-9becba83bbd3  TRNEHCV128F930CE23   
159378  B00004NH92  31d692d2-c9a5-3637-bc59-9becba83bbd3  TRXXOQB128F930CE17   
159379  B00004NH92  31d692d2-c9a5-3637-bc59-9becba83bbd3  TRQHKOT128F930CE20   
159380  B00004NH92  31d692d2-c9a5-3637-bc59-9becba83bbd3  TRTAIPQ128F930CE1B   
159381  B0008D5L4Y  51d88da7-6763-3530-8d7e-2b3f600558a5  TRJLVFG128F4285E8A   

                              recording

**Create a dataset with amazonids, recording mbids, and genres**

In [6]:
track_df_dataset = {"amazonIDs": track_dataset['amazon_id'], "recording_mbids": track_dataset['recording_mbid'], "genres": track_dataset['genres']}
track_df = pd.DataFrame(track_df_dataset)
track_df.head()

,amazonIDs,recording_mbids,genres
0,B0018ZB6ZO,66dd470f-4c4f-4caa-b8d4-2ebc80a83cb6,Dance & Electronic
1,B0018ZB6ZO,b45aef36-c3a3-42f2-90fb-728e8b3c54a2,Dance & Electronic
2,B000003HGR,36b15f42-b441-4d64-8ba9-ca264ed1c6f1,Alternative Rock
3,B000003HGR,279fabb3-830e-4d04-86d5-53668bc42bfa,Alternative Rock
4,B000003HGR,381b90e1-c07f-4a53-be8c-6a2107d7a17f,Alternative Rock


**Combine genres by amazonIDs**

In [7]:
track_df1 = track_df.groupby('amazonIDs')['genres'].apply(', '.join).reset_index()
track_df1.head()

,amazonIDs,genres
0,1458389375,"Jazz,Pop"
1,1591791065,"New Age,Dance Pop,World Music,Pop,Classical"
2,1906063443,"Europe,Christian,Eastern Europe,Pop,Gypsy,Worl..."
3,1929243766,"Comedy & Spoken Word,Pop, Comedy & Spoken Word..."
4,1930864159,"Pop & Contemporary,Christian,Gospel,Pop, Pop &..."


In [8]:
track_df1.shape

(31471, 2)

**Combine recording_mbids by amazonIDs**

In [9]:
track_df2 = track_df.groupby('amazonIDs')['recording_mbids'].apply(', '.join).reset_index()
track_df2.head()

,amazonIDs,recording_mbids
0,1458389375,3b288ece-524f-4531-8b6b-0148fa521ecd
1,1591791065,f5390cba-e72a-4f58-997b-825881f730da
2,1906063443,"139a8343-49ab-4f94-aafd-39494a3e707d, 8aefb4b4..."
3,1929243766,"effb0377-c903-4ac1-b982-c8c7248d709f, 91db51a7..."
4,1930864159,"7774e668-80a1-4d21-87f9-223850e1023d, f44ad78f..."


In [10]:
track_df2.shape #should be the same shape as track_df1

(31471, 2)

**Combine df1 and df2 to have a new dataset that groups recording mbids and genres by its amazon ids**

In [12]:
new_dataset = {"amazonIDs": track_df1['amazonIDs'], "recording_mbids": track_df2['recording_mbids'], "genres": track_df1['genres']}
track_df = pd.DataFrame(new_dataset)
track_df.head()

,amazonIDs,recording_mbids,genres
0,1458389375,3b288ece-524f-4531-8b6b-0148fa521ecd,"Jazz,Pop"
1,1591791065,f5390cba-e72a-4f58-997b-825881f730da,"New Age,Dance Pop,World Music,Pop,Classical"
2,1906063443,"139a8343-49ab-4f94-aafd-39494a3e707d, 8aefb4b4...","Europe,Christian,Eastern Europe,Pop,Gypsy,Worl..."
3,1929243766,"effb0377-c903-4ac1-b982-c8c7248d709f, 91db51a7...","Comedy & Spoken Word,Pop, Comedy & Spoken Word..."
4,1930864159,"7774e668-80a1-4d21-87f9-223850e1023d, f44ad78f...","Pop & Contemporary,Christian,Gospel,Pop, Pop &..."


In [13]:
track_df.to_csv('whole_audio_unprocessed_dataset.csv')

# Reduce audio dataset to have same albums as text dataset

**Load audio data from step above**

In [14]:
audio_df = pd.read_csv('whole_audio_unprocessed_dataset.csv')
audio_df = audio_df.drop(columns = ['Unnamed: 0'])

In [15]:
audio_df

,amazonIDs,recording_mbids,genres
0,1458389375,3b288ece-524f-4531-8b6b-0148fa521ecd,"Jazz,Pop"
1,1591791065,f5390cba-e72a-4f58-997b-825881f730da,"New Age,Dance Pop,World Music,Pop,Classical"
2,1906063443,"139a8343-49ab-4f94-aafd-39494a3e707d, 8aefb4b4...","Europe,Christian,Eastern Europe,Pop,Gypsy,Worl..."
3,1929243766,"effb0377-c903-4ac1-b982-c8c7248d709f, 91db51a7...","Comedy & Spoken Word,Pop, Comedy & Spoken Word..."
4,1930864159,"7774e668-80a1-4d21-87f9-223850e1023d, f44ad78f...","Pop & Contemporary,Christian,Gospel,Pop, Pop &..."
...,...,...,...
31466,B00JVSIMLQ,"aa54d86c-a1a7-46ca-a613-a2c38c7dfa69, 7603b1c0...","Latin Music, Latin Music, Latin Music, Latin M..."
31467,B00KDL9TTE,"a4ae86a2-b4cf-4e68-8800-566f11d1a8cf, 558ade05...","Pop,Easy Listening, Pop,Easy Listening, Pop,Ea..."
31468,B00KE5M5IQ,"74bbe3d0-c268-4426-84b7-d4997504bc4c, 82a802cb...","Latin Music, Latin Music, Latin Music, Latin M..."
31469,B00KG10ZXU,5fec9519-1e6f-4557-bbbf-3a7a42435ada,"Broadway & Vocalists,Vocal Pop,Pop"


**Load preprocessed text data and match amazon ids then sort them st amazon ids are in the same order for both datasets**

In [16]:
text_df = pd.read_pickle("df2000_babelfied.pkl")
entries_to_keep = text_df.amazonIDs.tolist()
audio_df = audio_df[audio_df['amazonIDs'].isin(entries_to_keep)]
text_df = text_df.sort_values(by=['amazonIDs'])

In [17]:
audio_df = audio_df.reset_index()
audio_df

,index,amazonIDs,recording_mbids,genres
0,2,1906063443,"139a8343-49ab-4f94-aafd-39494a3e707d, 8aefb4b4...","Europe,Christian,Eastern Europe,Pop,Gypsy,Worl..."
1,3,1929243766,"effb0377-c903-4ac1-b982-c8c7248d709f, 91db51a7...","Comedy & Spoken Word,Pop, Comedy & Spoken Word..."
2,4,1930864159,"7774e668-80a1-4d21-87f9-223850e1023d, f44ad78f...","Pop & Contemporary,Christian,Gospel,Pop, Pop &..."
3,6,3937406328,a2a5ca2b-96fb-4d7d-a96f-e60bd75a624b,"Europe,World Music,Blues Rock,Rock,Pop,Dance &..."
4,7,4266950926,"5947ac64-d026-47a2-a1ae-63ab597f1710, 10213ffb...","New Age,Meditation,World Music,Pop, New Age,Me..."
...,...,...,...,...
1995,2629,B000001AD0,3a653326-cf37-432b-9bde-fdbca8c21660,"Oldies,Dance & Electronic,Motown,Philly Soul,S..."
1996,2630,B000001AEI,"22669ef6-15f7-416f-a40e-40535a63cca5, 59ad5aa7...","Oldies,Dance & Electronic,Motown,Philly Soul,S..."
1997,2631,B000001AEK,f84737c3-0335-4d99-8f86-802151c26f46,"Oldies,Motown,Soul,Pop,R&B,Classic R&B,Girl Gr..."
1998,2633,B000001AFH,"9732b5c6-0b7d-4726-b90a-b01e08432820, cd14a070...","R&B,Blues,Motown,Soul,Classic R&B, R&B,Blues,M..."


In [18]:
audio_df.to_csv('audio_unprocessed_MLP.csv')

In [19]:
text_df = text_df.reset_index()

In [21]:
text_df

,index,amazonIDs,reviews,genres,clean_reviews
0,2,1906063443,"Contrary to the previous review, this IS a com...","Europe,Christian,Eastern Europe,Pop,Gypsy,Worl...",contrary previous review compilation single ar...
1,3,1929243766,My family and I all first experienced John on ...,"Comedy & Spoken Word,Pop",family first experienced john comedy central t...
2,4,1930864159,Awesome. This is so good when needing reminded...,"Pop & Contemporary,Christian,Gospel,Pop",awesome good needing reminded quiet hear god w...
3,6,3937406328,The purchase of this set was a bold step for m...,"Europe,World Music,Blues Rock,Rock,Pop,Dance &...",purchase set bold step dont boxed sets learned...
4,7,4266950926,I stumbled onto this CD one day at a listening...,"New Age,Meditation,World Music,Pop",stumbled onto cd one day listening post first ...
...,...,...,...,...,...
1995,2629,B000001AD0,"All time classic, hadn't played it in years un...","Oldies,Dance & Electronic,Motown,Philly Soul,S...",time classic hadnt played years recently given...
1996,2630,B000001AEI,"its vinyl, its jackson 5, whats not to love? t...","Oldies,Dance & Electronic,Motown,Philly Soul,S...",vinyl jackson whats love sleeve picture bit fa...
1997,2631,B000001AEK,my record arrived so brittle that it broke in ...,"Oldies,Motown,Soul,Pop,R&B,Classic R&B,Girl Gr...",record arrived brittle broke half packaging ma...
1998,2633,B000001AFH,"I have waited for this album, AN EVENING WITH ...","R&B,Blues,Motown,Soul,Classic R&B",waited evening diana ross released cd long tim...


In [22]:
text_df.to_csv('text_babelified_processed_MLP.csv')

# Preprocessing

In [4]:
audio_df = pd.read_csv('audio_unprocessed_MLP.csv')
audio_df = audio_df.drop(columns = ['Unnamed: 0'])
audio_df

,index,amazonIDs,recording_mbids,genres
0,2,1906063443,"139a8343-49ab-4f94-aafd-39494a3e707d, 8aefb4b4...","Europe,Christian,Eastern Europe,Pop,Gypsy,Worl..."
1,3,1929243766,"effb0377-c903-4ac1-b982-c8c7248d709f, 91db51a7...","Comedy & Spoken Word,Pop, Comedy & Spoken Word..."
2,4,1930864159,"7774e668-80a1-4d21-87f9-223850e1023d, f44ad78f...","Pop & Contemporary,Christian,Gospel,Pop, Pop &..."
3,6,3937406328,a2a5ca2b-96fb-4d7d-a96f-e60bd75a624b,"Europe,World Music,Blues Rock,Rock,Pop,Dance &..."
4,7,4266950926,"5947ac64-d026-47a2-a1ae-63ab597f1710, 10213ffb...","New Age,Meditation,World Music,Pop, New Age,Me..."
...,...,...,...,...
1995,2629,B000001AD0,3a653326-cf37-432b-9bde-fdbca8c21660,"Oldies,Dance & Electronic,Motown,Philly Soul,S..."
1996,2630,B000001AEI,"22669ef6-15f7-416f-a40e-40535a63cca5, 59ad5aa7...","Oldies,Dance & Electronic,Motown,Philly Soul,S..."
1997,2631,B000001AEK,f84737c3-0335-4d99-8f86-802151c26f46,"Oldies,Motown,Soul,Pop,R&B,Classic R&B,Girl Gr..."
1998,2633,B000001AFH,"9732b5c6-0b7d-4726-b90a-b01e08432820, cd14a070...","R&B,Blues,Motown,Soul,Classic R&B, R&B,Blues,M..."


In [5]:
total_useless_albums_Index = []
total_useless_albums_index = []

In [6]:
chroma_stft = []
spec_cent = []
spec_bw = []
rolloff = []
zcr = []
mfcc_1 = []
mfcc_2 = []
mfcc_3 = []
mfcc_4 = []
mfcc_5 = []
useless_albums_Index = []
useless_albums_index = []
    
for row in audio_df[800:900].itertuples():
    recording_mbids = row.recording_mbids
    split_recording_mbids = recording_mbids.split(', ')
    album_averages = get_Album_Averages(split_recording_mbids)
    
    if album_averages != []:
        chroma_stft.append(album_averages[0])
        spec_cent.append(album_averages[1])
        spec_bw.append(album_averages[2])
        rolloff.append(album_averages[3])
        zcr.append(album_averages[4])
        mfcc_1.append(album_averages[5])
        mfcc_2.append(album_averages[6])
        mfcc_3.append(album_averages[7])
        mfcc_4.append(album_averages[8])
        mfcc_5.append(album_averages[9])
        
    else:
        print("useless album:", row.Index)
        useless_albums_Index.append(row.Index)
        useless_albums_index.append(row.index)

Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
useless album: 801
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
useless album: 802
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Song not found
Preview not found
Preview not found
useless album: 804
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview not found
Preview no

TypeError: object of type 'NoneType' has no len()

In [134]:
print(len(chroma_stft))
print(len(spec_cent))
print(len(spec_bw))
print(len(rolloff))
print(len(zcr))
print(len(mfcc_1))
print(len(mfcc_2))
print(len(mfcc_3))
print(len(mfcc_4))
print(len(mfcc_5))

33
33
33
33
33
33
33
33
33
33


In [135]:
print(len(useless_albums_Index))
print(useless_albums_Index)
total_useless_albums_Index.append(useless_albums_Index)

19
[801, 802, 804, 808, 809, 810, 811, 815, 816, 818, 819, 820, 821, 839, 841, 848, 849, 850, 851]


In [136]:
print(len(useless_albums_index))
print(useless_albums_index)
total_useless_albums_index.append(useless_albums_index)

19
[1068, 1071, 1074, 1078, 1079, 1081, 1082, 1087, 1088, 1091, 1092, 1093, 1094, 1116, 1120, 1128, 1129, 1130, 1132]


In [125]:
dropped_audio_df = audio_df.drop(index = useless_albums_Index)
print(dropped_audio_df.shape)

(1957, 4)


In [126]:
dropped_audio_df = dropped_audio_df.drop(index = range(0,700))

In [127]:
dropped_audio_df.head(57)

,index,amazonIDs,recording_mbids,genres
701,930,B000000E7Q,"15f46385-6158-49d3-959b-90a2427731cf, 6869427d...","Europe,Irish Folk,Ireland,Celtic Folk,British ..."
702,931,B000000E7R,"8f58afc4-01be-4cb7-b5fe-16d1f2e40ebd, 6eff2dd5...","Folk Rock,British & Celtic Folk,Pop,Traditiona..."
703,932,B000000E7T,"8ae090a3-b52b-45e4-aa47-4a20e04c8707, a0857048...","Europe,Irish Folk,Ireland,British Isles,Britis..."
704,933,B000000E7V,"fc16b5be-1034-42e1-b573-df14423f7fd7, a906b6d0...","Folk Rock,Contemporary Folk,Jazz,British & Cel..."
705,934,B000000E84,"d5028e3d-4ad0-4cf5-97cd-44a51f8ec45a, 3411727c...","Folk Rock,British & Celtic Folk,Pop,Traditiona..."
706,935,B000000E8B,"3f1f3436-2fab-464f-8daf-72e82e90283f, 5f3be397...","Classic Country,Country,British & Celtic Folk,..."
707,936,B000000E8J,"5ce58b23-4585-49b6-8bbe-14980856be1e, 5deae18a...","Europe,Irish Folk,Ireland,British Isles,Britis..."
708,937,B000000E8N,"e6615725-3d45-4d55-b95f-dd690fb53f15, 2fc696b2...","Europe,Irish Folk,Contemporary Folk,Ireland,Br..."
709,939,B000000E8S,"ab37229a-da71-4a5e-9f49-d6845b09d717, 0b0dd2e5...","Folk Rock,British & Celtic Folk,Pop,Traditiona..."
710,941,B000000E8X,199caf29-82c1-4815-ad4f-a00ebc4e1913,"Folk Rock,British & Celtic Folk,Pop,Traditiona..."


In [128]:
dropped_audio_df = dropped_audio_df.reset_index(drop = True)

In [129]:
dropped_audio_df.head(57)

,index,amazonIDs,recording_mbids,genres
0,930,B000000E7Q,"15f46385-6158-49d3-959b-90a2427731cf, 6869427d...","Europe,Irish Folk,Ireland,Celtic Folk,British ..."
1,931,B000000E7R,"8f58afc4-01be-4cb7-b5fe-16d1f2e40ebd, 6eff2dd5...","Folk Rock,British & Celtic Folk,Pop,Traditiona..."
2,932,B000000E7T,"8ae090a3-b52b-45e4-aa47-4a20e04c8707, a0857048...","Europe,Irish Folk,Ireland,British Isles,Britis..."
3,933,B000000E7V,"fc16b5be-1034-42e1-b573-df14423f7fd7, a906b6d0...","Folk Rock,Contemporary Folk,Jazz,British & Cel..."
4,934,B000000E84,"d5028e3d-4ad0-4cf5-97cd-44a51f8ec45a, 3411727c...","Folk Rock,British & Celtic Folk,Pop,Traditiona..."
5,935,B000000E8B,"3f1f3436-2fab-464f-8daf-72e82e90283f, 5f3be397...","Classic Country,Country,British & Celtic Folk,..."
6,936,B000000E8J,"5ce58b23-4585-49b6-8bbe-14980856be1e, 5deae18a...","Europe,Irish Folk,Ireland,British Isles,Britis..."
7,937,B000000E8N,"e6615725-3d45-4d55-b95f-dd690fb53f15, 2fc696b2...","Europe,Irish Folk,Contemporary Folk,Ireland,Br..."
8,939,B000000E8S,"ab37229a-da71-4a5e-9f49-d6845b09d717, 0b0dd2e5...","Folk Rock,British & Celtic Folk,Pop,Traditiona..."
9,941,B000000E8X,199caf29-82c1-4815-ad4f-a00ebc4e1913,"Folk Rock,British & Celtic Folk,Pop,Traditiona..."


In [130]:
audio_processed = dropped_audio_df[0:57]
print(audio_processed)

    index   amazonIDs                                    recording_mbids  \
0     930  B000000E7Q  15f46385-6158-49d3-959b-90a2427731cf, 6869427d...   
1     931  B000000E7R  8f58afc4-01be-4cb7-b5fe-16d1f2e40ebd, 6eff2dd5...   
2     932  B000000E7T  8ae090a3-b52b-45e4-aa47-4a20e04c8707, a0857048...   
3     933  B000000E7V  fc16b5be-1034-42e1-b573-df14423f7fd7, a906b6d0...   
4     934  B000000E84  d5028e3d-4ad0-4cf5-97cd-44a51f8ec45a, 3411727c...   
5     935  B000000E8B  3f1f3436-2fab-464f-8daf-72e82e90283f, 5f3be397...   
6     936  B000000E8J  5ce58b23-4585-49b6-8bbe-14980856be1e, 5deae18a...   
7     937  B000000E8N  e6615725-3d45-4d55-b95f-dd690fb53f15, 2fc696b2...   
8     939  B000000E8S  ab37229a-da71-4a5e-9f49-d6845b09d717, 0b0dd2e5...   
9     941  B000000E8X               199caf29-82c1-4815-ad4f-a00ebc4e1913   
10    943  B000000E93  c0a96782-2e63-42b2-9963-e5fd7793a0df, 505f57a8...   
11    944  B000000E98  1d9f581c-550e-4636-9147-9147a43236c6, 9135a27a...   
12    945  B

In [131]:
#print(track_Name)
dataset = {"index": audio_processed['index'], "amazonIDs": audio_processed['amazonIDs'], "Chroma Stft": chroma_stft, "Spectral Centroid": spec_cent, "Spectral Bandwidth": spec_bw, "Spectral Rolloff": rolloff, "ZCR": zcr, "MFCC 1": mfcc_1, "MFCC 2": mfcc_2, "MFCC 3": mfcc_3, "MFCC 4": mfcc_4, "MFCC 5": mfcc_5, "Genres": audio_processed['genres']}
df = pd.DataFrame(dataset)
df.head()

,index,amazonIDs,Chroma Stft,Spectral Centroid,Spectral Bandwidth,Spectral Rolloff,ZCR,MFCC 1,MFCC 2,MFCC 3,MFCC 4,MFCC 5,Genres
0,930,B000000E7Q,0.181654,1405.886231,1562.861033,2504.531522,0.070848,-187.619449,129.308173,-27.638362,7.341697,-17.777177,"Europe,Irish Folk,Ireland,Celtic Folk,British ..."
1,931,B000000E7R,0.364005,2142.524192,2436.162779,4883.309360,0.090127,-116.764107,101.228018,6.424787,27.106144,-0.290760,"Folk Rock,British & Celtic Folk,Pop,Traditiona..."
2,932,B000000E7T,0.263264,1982.215416,2125.601347,3918.117974,0.105639,-198.329065,101.653726,-15.884568,21.044567,2.748388,"Europe,Irish Folk,Ireland,British Isles,Britis..."
3,933,B000000E7V,0.256842,1791.164852,2152.651627,3538.099439,0.059010,-439.495466,70.764704,10.646045,14.773071,-0.776057,"Folk Rock,Contemporary Folk,Jazz,British & Cel..."
4,934,B000000E84,0.275450,1815.175429,1966.905636,3551.817412,0.086263,-243.182638,106.485731,-11.194514,30.671065,-8.035985,"Folk Rock,British & Celtic Folk,Pop,Traditiona..."


In [132]:
df.to_csv('audio_processed_MLP_h.csv')